# Module 3. Training on SageMaker Environment with Profiling Enabled
---

This notebook can train on a single GPU or perform distributed training using PyTorch DDP.

This hands-on can be completed in about **20 minutes**. 

<div class="alert alert-info"><h4>Note</h4><p>
If you are considering training large models & hundreds of gigabytes to terabytes of data, consider the SageMaker Distributed Training option.</p></div>

In [18]:
%store -r 
%store

Stored variables and their in-db values:
base_model_name              -> 'mobilenetv2'
bucket                       -> 'sagemaker-us-east-1-143656149352'
classes                      -> ['brown_normal_chinese', 'red_abnormal_korean', 'b
dataset_dir                  -> 'smartfactory'
local_model_path             -> '/opt/ml/model'
model_name                   -> 'model_best.pth'
num_classes                  -> 5
raw_dir                      -> 'raw'


In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import copy
import time
import numpy as np
import torch, os
import matplotlib.pyplot as plt
import src.train_utils

import boto3
import sagemaker

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
region = boto3.Session().region_name

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
try:
    bucket 
    dataset_dir        
    print("[OK] You can proceed.")
except NameError:
    print("+"*60)
    print("[ERROR] Please run '01_make_augmented_imgs.ipynb' before you continue.")
    print("+"*60)

[OK] You can proceed.


In [21]:
from sagemaker.pytorch import PyTorch
#role = sagemaker.get_execution_role()
role = "arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20210315T163355"
s3_path = f's3://{bucket}/{dataset_dir}'

In [22]:
# import wandb
# os.environ["WANDB_NOTEBOOK_NAME"] = dataset_dir
# wandb.sagemaker_auth(path="./src")

<br>

# 1. SageMaker Debugger Profiling Enabled
---

To enable profiling, create a `ProfilerConfig` object and pass it to the SageMaker estimator `profiler_config` parameter. This example sets the profiling interval to 500 milliseconds (0.5 seconds).

In [23]:
from sagemaker.debugger import (ProfilerConfig,
                                FrameworkProfile,
                                CollectionConfig,
                                DebuggerHookConfig,
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig,
                                Rule,
                                PythonProfiler,
                                cProfileTimer,
                                ProfilerRule,
                                rule_configs)

# Location in S3 where the debugger output will be stored is mentioned in the previous step

# Set the profile config for both system and framework metrics
profiler_config = ProfilerConfig(
    system_monitor_interval_millis = 500,
    framework_profile_params = FrameworkProfile(
        detailed_profiling_config = DetailedProfilingConfig(
            start_step = 5, 
            num_steps = 10
        ),
        dataloader_profiling_config = DataloaderProfilingConfig(
            start_step = 7, 
            num_steps = 10
        ),
        python_profiling_config = PythonProfilingConfig(
            start_step = 9, 
            num_steps = 10,
            python_profiler = PythonProfiler.CPROFILE, 
            cprofile_timer = cProfileTimer.TOTAL_TIME
        )
    )
)

# Set the debugger hook config to save tensors
debugger_hook_config = DebuggerHookConfig(
    collection_configs = [
        CollectionConfig(name = 'weights'),
        CollectionConfig(name = 'gradients')
    ]
)

# Set the rules to analyze tensors emitted during training. 
# These specific set of rules will inspect the overall training performance and progress of the model
rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.stalled_training_rule())
]

<br>

# 2. Run a Training Job
---

Create a training job using the standard SageMaker Estimator API for PyTorch. 

The python training script file utilized by SageMaker will also use the same python file as the on-premise. Therefore, if you add only SageMaker environment variables, you can train without modifying the source code. The code snippet below is an example of setting SageMaker environment variables.

```python
parser = argparse.ArgumentParser()
...

# SageMaker environment variables
parser.add_argument('--hosts', type=list,
                    default=json.loads(os.environ['SM_HOSTS']))
parser.add_argument('--current_host', type=str,
                    default=os.environ['SM_CURRENT_HOST'])
parser.add_argument('--model_dir', type=str,
                    default=os.environ['SM_MODEL_DIR'])
parser.add_argument('--model_chkpt_dir', type=str,
                    default='/opt/ml/checkpoint')    
parser.add_argument('--train_dir', type=str,
                    default=os.environ['SM_CHANNEL_TRAIN'])
parser.add_argument('--valid_dir', type=str,
                    default=os.environ['SM_CHANNEL_VALID'])    
parser.add_argument('--num_gpus', type=int,
                    default=os.environ['SM_NUM_GPUS'])
parser.add_argument('--output_data_dir', type=str,
                    default=os.environ.get('SM_OUTPUT_DATA_DIR'))
```

For example, among the environment variables, `SM_MODEL_DIR` means `/opt/ml/model` in the SageMaker training container environment. For various environment variables, please refer to [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit).

In [30]:
IS_DISTRIBUTED_TRAINING = True
USE_WANDB = False

entry_point = 'train_single_gpu_wandb.py' if USE_WANDB else 'train_single_gpu.py'

if IS_DISTRIBUTED_TRAINING:
    estimator = PyTorch(entry_point='train_multi_gpu.py',
                        source_dir='./src',
                        role=role,
                        instance_type='ml.p3.8xlarge',
                        instance_count=1,
                        framework_version='1.6.0',
                        py_version='py3',
                        disable_profiler=False,
                        profiler_config=profiler_config,  
                        debugger_hook_config=debugger_hook_config,
                        hyperparameters = {'num_epochs': 10, 
                                           'batch_size': 128, # This parameter is divided by the number of GPUs
                                           'lr': 0.0005,
                                          }
                       )
else:
    estimator = PyTorch(entry_point=entry_point,
                        source_dir='./src',
                        role=role,
                        instance_type='ml.g4dn.xlarge',
                        instance_count=1,
                        framework_version='1.6.0',
                        py_version='py3',
                        disable_profiler=False,
                        profiler_config=profiler_config,  
                        debugger_hook_config=debugger_hook_config,                        
                        hyperparameters = {'num_epochs': 10, 
                                           'batch_size': 64,
                                           'lr': 0.0005,
                                          }     
                       )
    
train_input = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, dataset_dir))  
valid_input = sagemaker.TrainingInput(s3_data='s3://{}/{}/valid'.format(bucket, dataset_dir))      

In [31]:
%%time
estimator.fit({'train': train_input, 'valid': valid_input}, wait=False)
train_job_name = estimator.latest_training_job.job_name

CPU times: user 782 ms, sys: 88.1 ms, total: 870 ms
Wall time: 1.62 s


In [32]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, train_job_name
        )
    )
)

In [33]:
display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, train_job_name
        )
    )
)

In [34]:
display(HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, train_job_name, region
        )
    )
)

If you set the wait argument of the `fit(.)` function to False, you can execute the code cell below to change it to a synchronous way to wait until training is complete.

In [35]:
sagemaker.Session().logs_for_job(job_name=train_job_name, wait=True)

2021-07-31 04:17:14 Starting - Starting the training job...
2021-07-31 04:17:45 Starting - Launching requested ML instancesProfilerReport-1627705032: InProgress
............
2021-07-31 04:19:37 Starting - Preparing the instances for training.........
2021-07-31 04:20:58 Downloading - Downloading input data...
2021-07-31 04:21:47 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-31 04:22:55,250 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-31 04:22:55,293 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-07-31 04:22:58,373 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-07-31 04:22:59,251 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Created wh

<br>
    
# 3. Analysis profiled data
---

When the training job starts, SageMaker Debugger starts collecting system and framework metrics. Once metrics collection begins, the profiling data can be analyzed in various ways, including plot and query.

In [36]:
rule_output_path = estimator.output_path + train_job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/rule-output


## Get Profiler Report

The `ProfilerReport()` rule generates an html report `profiler-report.html` with a summary of the basic rules and recommendations for next steps. You can find this report in your S3 bucket.

For more information on how to download and open the Debugger Profiling Report, see '[SageMaker Debugger Profiling Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html) in the SageMaker Developer Guide.

**[Caution] If running in JupyterLab, click "Trust HTML" at the top left of the screen to display the html report normally!**

In [37]:
import src.profiling_utils as profiling_utils
import json, os
from IPython.core.display import display, HTML

rule_output_path = estimator.output_path + train_job_name + "/rule-output"
output_dir = './output'
profile_output = output_dir+'/ProfilerReport'
profile_report_folder = profiling_utils.get_profile_report_folder(bucket, train_job_name + "/rule-output")

!rm -rf $output_dir

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(profile_output):
    os.makedirs(profile_output)    
    
!aws s3 ls {rule_output_path}/{profile_report_folder}/profiler-output/
!aws s3 cp {rule_output_path}/{profile_report_folder}/profiler-output/ {output_dir}/ProfilerReport/ --recursive    

display(HTML('<b>ProfilerReport : <a href="{}profiler-report.html">Profiler Report</a></b>'.format(output_dir+"/ProfilerReport/")))

[2021-07-31 04:41:08.183 ip-172-16-189-220:116274 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


                           PRE profiler-reports/
2021-07-31 04:26:59     368813 profiler-report.html
2021-07-31 04:26:59     223763 profiler-report.ipynb
download: s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/rule-output/ProfilerReport-1627705032/profiler-output/profiler-reports/Dataloader.json to output/ProfilerReport/profiler-reports/Dataloader.json
download: s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/rule-output/ProfilerReport-1627705032/profiler-output/profiler-reports/IOBottleneck.json to output/ProfilerReport/profiler-reports/IOBottleneck.json
download: s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/rule-output/ProfilerReport-1627705032/profiler-output/profiler-reports/LoadBalancing.json to output/ProfilerReport/profiler-reports/LoadBalancing.json
download: s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/rule-output/ProfilerReport-1627705032/profile

In [38]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
tj = TrainingJob(train_job_name, region)

# Retrieve a description of the training job description and the S3 bucket URI where the metric data are saved
tj.describe_training_job()
tj.get_config_and_profiler_s3_output_path()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-143656149352/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 7, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 5, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 9, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760'}}
s3 path:s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/profiler-output


({'S3OutputPath': 's3://sagemaker-us-east-1-143656149352/',
  'ProfilingIntervalInMilliseconds': 500,
  'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 7, "NumSteps": 10, "MetricsRegex": ".*", }',
   'DetailedProfilingConfig': '{"StartStep": 5, "NumSteps": 10, }',
   'FileOpenFailThreshold': '50',
   'LocalPath': '/opt/ml/output/profiler',
   'PythonProfilingConfig': '{"StartStep": 9, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }',
   'RotateFileCloseIntervalInSeconds': '60',
   'RotateMaxFileSizeInBytes': '10485760'}},
 's3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/profiler-output')

In [39]:
# Wait for the data to be available
tj.wait_for_sys_profiling_data_to_be_available()
tj.wait_for_framework_profiling_data_to_be_available()



Profiler data from system is available
[2021-07-31 04:42:20.190 ip-172-16-189-220:116274 INFO algorithm_metrics_reader.py:192] S3AlgorithmMetricsReader created with bucket:sagemaker-us-east-1-143656149352 and prefix:pytorch-training-2021-07-31-04-17-12-122/profiler-output/framework/


 Profiler data from framework is available
Found recorded framework annotations. Latest available timestamp microsseconds_since_epoch is:1627705587086310 , human_readable_timestamp in utc: 2021-07-31T04:26:27:086310


## Get Metrics Dataframe

In [40]:
%%time
system_metrics_df, framework_metrics_df = profiling_utils.get_profiling_df(tj)

display(system_metrics_df.head())
display(framework_metrics_df.head())

[2021-07-31 04:42:21.466 ip-172-16-189-220:116274 INFO algorithm_metrics_reader.py:192] S3AlgorithmMetricsReader created with bucket:sagemaker-us-east-1-143656149352 and prefix:pytorch-training-2021-07-31-04-17-12-122/profiler-output/framework/
[2021-07-31 04:42:21.549 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 7 event files
[2021-07-31 04:42:23.295 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 20 event files
[2021-07-31 04:42:29.706 ip-172-16-189-220:116274 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1627705402105968
[2021-07-31 04:42:30.624 ip-172-16-189-220:116274 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1627705402105885
[2021-07-31 04:42:31.400 ip-172-16-189-220:116274 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1627705402102903
[2021-07-31 04:42:32.197 ip-172-16-189-220:116274 INFO trace_event_file_parser.py:197] Start time for events in uSeconds = 1

,timestamp,timestamp_us,value,system_metric,dimension,nodeID,type
0,2021-07-31T04:20:47:685272,47685272,0.0,gpu3,GPUMemoryUtilization,algo-1,gpu
1,2021-07-31T04:20:47:685272,47685272,0.0,gpu0,GPUUtilization,algo-1,gpu
2,2021-07-31T04:20:47:685272,47685272,0.0,gpu0,GPUMemoryUtilization,algo-1,gpu
3,2021-07-31T04:20:47:685272,47685272,0.0,gpu1,GPUUtilization,algo-1,gpu
4,2021-07-31T04:20:47:685272,47685272,0.0,gpu1,GPUMemoryUtilization,algo-1,gpu


,start_time,end_time,start_time_us,end_time_us,tid,pid,framework_metric,step,bytes,process,nodeID
0,2021-07-31T04:23:22:238430,2021-07-31T04:23:25:120317,202238430,205120317,-1453395699472749472,93,Step:ModeKeys.GLOBAL,0,-1,Step:ModeKeys.GLOBAL,algo-1-3
1,2021-07-31T04:23:25:404894,2021-07-31T04:23:25:448978,205404894,205448978,-1453395699472749472,93,Step:ModeKeys.GLOBAL,1,-1,Step:ModeKeys.GLOBAL,algo-1-3
2,2021-07-31T04:23:25:587714,2021-07-31T04:23:25:614453,205587714,205614453,-1453395699472749472,93,Step:ModeKeys.GLOBAL,2,-1,Step:ModeKeys.GLOBAL,algo-1-3
3,2021-07-31T04:23:25:770143,2021-07-31T04:23:25:796793,205770143,205796793,-1453395699472749472,93,Step:ModeKeys.GLOBAL,3,-1,Step:ModeKeys.GLOBAL,algo-1-3
4,2021-07-31T04:23:25:951922,2021-07-31T04:23:25:986116,205951922,205986116,-1453395699472749472,93,Step:ModeKeys.GLOBAL,4,-1,Step:ModeKeys.GLOBAL,algo-1-3


CPU times: user 11.6 s, sys: 688 ms, total: 12.3 s
Wall time: 18.5 s


## Plot profiling metrics

In [41]:
%%time
profiling_utils.plot_profiling_metrics(tj)

[2021-07-31 04:42:40.135 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 7 event files
Found 49929 system metrics events from timestamp_in_us:0 to timestamp_in_us:1627705560000000
select events:['total']
select dimensions:['CPU', 'GPU', 'I/O']
filtered_events:{'total'}
filtered_dimensions:{'I/OWaitPercentage-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1'}


filtered_dimensions:{'I/OWaitPercentage-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1'}
[2021-07-31 04:42:41.304 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 7 event files
select events:['total']
select dimensions:['CPU', 'GPU', 'I/O']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization', 'GPUUtilization', 'I/OWaitPercentage', 'GPUMemoryUtilization'}


StepHistogram created, last_timestamp found:1627705587086310
stephistogram getting events from 1627705582086310 to 1627705587086310
[2021-07-31 04:42:42.142 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 20 event files
Total events fetched:2568
Select metrics:['Step:ModeKeys', 'Forward-node', 'Backward\\(post-forward\\)-node']
Available_metrics: ['Forward-SubModuleInternal-nodeid:algo-1-92', 'Forward-nodeid:algo-1-92', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-92', 'Forward-SubModuleInternal-nodeid:algo-1-90', 'Forward-nodeid:algo-1-90', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-90', 'Forward-SubModuleInternal-nodeid:algo-1-93', 'Forward-nodeid:algo-1-93', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-93', 'Forward-SubModuleInternal-nodeid:algo-1-91', 'Forward-nodeid:algo-1-91', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-91']
Filtered metrics:['Step:ModeKeys.GLOBAL-nodeid:algo-1-92', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-90', 'Step:ModeKeys.GLOBAL-nodeid:algo-1-93', 'Step:ModeKeys.GLOBAL-nodeid:al

select events:['total']
select dimensions:['CPU', 'GPU', 'I/O']
filtered_events:{'total'}
filtered_dimensions:{'I/OWaitPercentage-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'CPUUtilization-nodeid:algo-1', 'GPUMemoryUtilization-nodeid:algo-1'}


[2021-07-31 04:42:57.423 ip-172-16-189-220:116274 INFO metrics_reader_base.py:134] Getting 20 event files


CPU times: user 33.2 s, sys: 1.71 s, total: 34.9 s
Wall time: 42.2 s


<br>

# 4. Import model artifacts from S3 into your Local Environment
---

In [42]:
import os
local_model_path = 'model'
model_name = 'model_best.pth'
base_model_name = 'mobilenetv2'
os.makedirs(local_model_path, exist_ok=True)
s3_model_path = estimator.model_data

In [43]:
%%bash -s "$local_model_path" "$s3_model_path"
aws s3 cp $2 $1
cd $1
tar -xzvf model.tar.gz
rm model.tar.gz

download: s3://sagemaker-us-east-1-143656149352/pytorch-training-2021-07-31-04-17-12-122/output/model.tar.gz to model/model.tar.gz
model_best.pth


In [44]:
%store train_job_name s3_model_path base_model_name local_model_path model_name

Stored 'train_job_name' (str)
Stored 's3_model_path' (str)
Stored 'base_model_name' (str)
Stored 'local_model_path' (str)
Stored 'model_name' (str)


<br>

# Next Step

In this session, the model was trained by invoking the SageMaker Training job. Please proceed to `04_neo_compile_deploy.ipynb`.